<a href="https://colab.research.google.com/github/ndn1954/CohereRAGEmbedChatbot/blob/main/Cohere_RAG_voicebot_with_whisper_Gardio_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cohere RAG chatbot for PDF with speech as input and speech as output

In [ ]:
!pip install langchain
!pip install pypdf
!pip install cohere
!pip install qdrant-client
!pip install gradio
!pip install langchain-community
!pip install sentence-transformers
!pip install faiss-cpu

In [ ]:
#Import Libraries
import os
import langchain
import gradio as gr


In [ ]:
os.environ['COHERE_API_KEY'] =  ''    #Set the Cohere API Key

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
DATA_PATH = "./"
DB_FAISS_PATH = "./"

loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                chunk_overlap=50)
texts = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                    model_kwargs={'device': 'cpu'})

db = FAISS.from_documents(texts, embeddings)
db.save_local(DB_FAISS_PATH)

In [ ]:
from langchain import PromptTemplate

In [ ]:
"""
Prompt template for QA retrieval for each vectorstore
"""
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])


In [ ]:
import langchain
langchain.debug = False
from langchain.chains import RetrievalQA
from langchain.llms import Cohere
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0.9),
                                 chain_type="stuff",
                                 retriever=db.as_retriever(),
                                 chain_type_kwargs={'prompt': prompt},
                                 return_source_documents=True)

In [ ]:
questions = [
           "question 1 ?",
           "question 2 ?"
           ]

In [ ]:
import datetime
# Test if RAG bot is working with cohere
for question in questions:
  starttime = datetime.datetime.now()
  answer = qa({"query": question})
  print(answer)
  endtime = datetime.datetime.now()
  print('Time taken for each qstn: ', endtime-starttime)

### Gradio app to support voice bot

In [ ]:
!pip install -q gradio torch torchaudio transformers
!pip install gTTS

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
import gtts

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    return transcriber({"sampling_rate": sr, "raw": y})["text"]  # type: ignore

def create_audio(txt):
  tts = gtts.gTTS(txt)
  # save the audio file
  tts.save("output.mp3")
  return "output.mp3"

def capture_qstn(audio):
    starttime = datetime.datetime.now()
    sr, data = audio
    txt = transcribe(audio)
    print('qstn:',txt)
    ans = qa(txt)
    print('ans:',ans)
    rag_endtime=datetime.datetime.now()
    print('Time taken at the end of RAG: ', rag_endtime-starttime)
    audi = create_audio(ans['result'])
    endtime = datetime.datetime.now()
    print('Time taken for each qstn: ', endtime-starttime)
    return audi

input_audio = gr.Audio(
    sources=["microphone"],
    waveform_options=gr.WaveformOptions(
        waveform_color="#01C6FF",
        waveform_progress_color="#0066B4",
        skip_length=2,
        show_controls=False,
    ),
)

demo = gr.Interface(
    fn=capture_qstn,
    inputs=input_audio,
    outputs="audio",
)

if __name__ == "__main__":
    demo.launch(debug=True)